In [8]:
import itertools
from collections import defaultdict
import os
import pickle
import multiprocessing

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import scipy
from scipy import stats
from statsmodels.stats.power import TTestIndPower
from tqdm import tqdm

from utils import filter_pairs, test_all

In [9]:
lp_source = 'ARA'
lp_target = 'ENU'
lengths = [600, 1200, 1800, 2400, 3000]

In [10]:
pairs = filter_pairs(lp_source, lp_target, size=500)
len(pairs)

Annotated data loaded


71

In [11]:
def simulate_fixed_pair(sys1, sys2, length=200, trials=1000, p=0.05):
    pvalues, lengths = [], []
    
    for i in range(0, trials):
        baseline_sys = sys1.sample(length, replace=True)
        other_sys = sys2.sample(length, replace=True)
        ind, pvalue = stats.mannwhitneyu(baseline_sys, other_sys)
        
        pvalues.append(pvalue)
        lengths.append(length)
    
    power = (np.array(pvalues) < p).sum() / trials
    
    return power, np.mean(lengths)

In [6]:
for length in lengths:
    results = test_all(lp_pairs, lambda x, y: simulate_fixed_pair(x, y, length=length, trials=1000))
    np.save(open('fixed_%s-%s_%d.npz' % (lp_source, lp_target, length), 'wb'), results)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [36:59<00:00,  4.44s/it]
